# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 9:20AM,244349,10,PRF-35280,Bio-PRF,Released
1,Aug 17 2022 9:13AM,244345,20,8414205,"ACI Healthcare USA, Inc.",Released
2,Aug 17 2022 9:12AM,244346,12,HH-36334,Hush Hush,Released
3,Aug 17 2022 9:12AM,244346,12,HH-36434,Hush Hush,Released
4,Aug 17 2022 9:12AM,244346,12,HH-36435,Hush Hush,Released
5,Aug 17 2022 9:12AM,244346,12,HH-36436,Hush Hush,Released
6,Aug 17 2022 9:12AM,244346,12,HH-36437,Hush Hush,Released
7,Aug 17 2022 9:12AM,244346,12,HH-36438,Hush Hush,Released
8,Aug 17 2022 9:12AM,244346,12,HH-36439,Hush Hush,Released
9,Aug 17 2022 9:12AM,244346,12,HH-36440,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,244341,Released,4
40,244343,Released,5
41,244345,Released,1
42,244346,Released,13
43,244349,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244341,NaN,NaN,4.0
244343,NaN,NaN,5.0
244345,NaN,NaN,1.0
244346,NaN,NaN,13.0
244349,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244222,9.0,0.0,3.0
244256,0.0,0.0,25.0
244259,0.0,9.0,3.0
244261,0.0,0.0,1.0
244268,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244222,9,0,3
244256,0,0,25
244259,0,9,3
244261,0,0,1
244268,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244222,9,0,3
1,244256,0,0,25
2,244259,0,9,3
3,244261,0,0,1
4,244268,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244222,9,,3
1,244256,,,25
2,244259,,9,3
3,244261,,,1
4,244268,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc."
2,Aug 17 2022 9:12AM,244346,12,Hush Hush
15,Aug 17 2022 9:06AM,244343,10,Emerginnova
20,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc."
25,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample
26,Aug 17 2022 8:49AM,244306,20,"ACI Healthcare USA, Inc."
27,Aug 17 2022 8:48AM,244341,10,"CLINUVEL, Inc."
31,Aug 17 2022 8:37AM,244337,20,"Exact-Rx, Inc."
32,Aug 17 2022 8:32AM,244336,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF,,,1
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",,,1
2,Aug 17 2022 9:12AM,244346,12,Hush Hush,,,13
3,Aug 17 2022 9:06AM,244343,10,Emerginnova,,,5
4,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",,,5
5,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample,,,1
6,Aug 17 2022 8:49AM,244306,20,"ACI Healthcare USA, Inc.",,,1
7,Aug 17 2022 8:48AM,244341,10,"CLINUVEL, Inc.",,,4
8,Aug 17 2022 8:37AM,244337,20,"Exact-Rx, Inc.",,,1
9,Aug 17 2022 8:32AM,244336,10,ISDIN Corporation,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF,1,,
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 17 2022 9:12AM,244346,12,Hush Hush,13,,
3,Aug 17 2022 9:06AM,244343,10,Emerginnova,5,,
4,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",5,,
5,Aug 17 2022 8:49AM,244340,10,TherapeuticsMD Elli Sample,1,,
6,Aug 17 2022 8:49AM,244306,20,"ACI Healthcare USA, Inc.",1,,
7,Aug 17 2022 8:48AM,244341,10,"CLINUVEL, Inc.",4,,
8,Aug 17 2022 8:37AM,244337,20,"Exact-Rx, Inc.",1,,
9,Aug 17 2022 8:32AM,244336,10,ISDIN Corporation,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF,1,,
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 17 2022 9:12AM,244346,12,Hush Hush,13,,
3,Aug 17 2022 9:06AM,244343,10,Emerginnova,5,,
4,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF,1.0,NaN,NaN
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Aug 17 2022 9:12AM,244346,12,Hush Hush,13.0,NaN,NaN
3,Aug 17 2022 9:06AM,244343,10,Emerginnova,5.0,NaN,NaN
4,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 9:20AM,244349,10,Bio-PRF,1.0,0.0,0.0
1,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Aug 17 2022 9:12AM,244346,12,Hush Hush,13.0,0.0,0.0
3,Aug 17 2022 9:06AM,244343,10,Emerginnova,5.0,0.0,0.0
4,Aug 17 2022 8:56AM,244307,20,"ACI Healthcare USA, Inc.",5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4397876,136.0,32.0,0.0
102,244320,1.0,0.0,0.0
12,488644,13.0,1.0,1.0
15,244222,3.0,0.0,9.0
16,977112,4.0,0.0,0.0
17,488643,2.0,0.0,0.0
19,244328,31.0,11.0,0.0
20,1710078,36.0,10.0,0.0
21,732959,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4397876,136.0,32.0,0.0
1,102,244320,1.0,0.0,0.0
2,12,488644,13.0,1.0,1.0
3,15,244222,3.0,0.0,9.0
4,16,977112,4.0,0.0,0.0
5,17,488643,2.0,0.0,0.0
6,19,244328,31.0,11.0,0.0
7,20,1710078,36.0,10.0,0.0
8,21,732959,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,136.0,32.0,0.0
1,102,1.0,0.0,0.0
2,12,13.0,1.0,1.0
3,15,3.0,0.0,9.0
4,16,4.0,0.0,0.0
5,17,2.0,0.0,0.0
6,19,31.0,11.0,0.0
7,20,36.0,10.0,0.0
8,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,136.0
1,102,Released,1.0
2,12,Released,13.0
3,15,Released,3.0
4,16,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0
Executing,32.0,0.0,1.0,0.0,0.0,0.0,11.0,10.0,0.0
Released,136.0,1.0,13.0,3.0,4.0,2.0,31.0,36.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21
0,Completed,0.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,32.0,0.0,1.0,0.0,0.0,0.0,11.0,10.0,0.0
2,Released,136.0,1.0,13.0,3.0,4.0,2.0,31.0,36.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21
0,Completed,0.0,0.0,1.0,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,32.0,0.0,1.0,0.0,0.0,0.0,11.0,10.0,0.0
2,Released,136.0,1.0,13.0,3.0,4.0,2.0,31.0,36.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()